In [89]:
import gensim
from IPython.display import clear_output
import json
import re
import random
import time
import pickle
from IPython.core.display import display, HTML
import locale
locale.setlocale(locale.LC_ALL, 'de_DE.utf-8')

# used for loading or saving
model_file = '/home/ramin/projects/ECO/src/python/modelbuilder/parsed_v3_valid.doc2vec'

In [90]:
# 2 Build sentence list (each sentence needs at least 1 tag)
filename = '/home/marcel/drive/data/eco/NAIL_DATAFIELD_txt/parsed_v3/parsed_v3_valid.txt'

sentences = []
from random import shuffle

for uid, line in enumerate(open(filename)):
    ls = gensim.models.doc2vec.LabeledSentence(words=line.split(), tags=['SENT_%s' % uid])
    sentences.append(ls)
print(len(sentences),'sentences')

(4266193, 'sentences')


In [91]:
# 3 TRAINING OR LOADING the doc2vec model and save it
# ALTERNATIVE: LOAD THE MODEL IN THE NEXT CELL

# tutorial https://rare-technologies.com/doc2vec-tutorial/
# proposes shuffling or learning reate adjustment. we gonna do both
# in total 20 epochs
# took ca. 6.30 hours

# FOR SAFETY REASON, BUILD ONLY WHEN FLAG IS SET

train_model = False
do_tests = True

if train_model:
    model = gensim.models.Doc2Vec(alpha=0.025, min_alpha=0.025)  # use fixed learning rate
    print('building vocab') 
    model.build_vocab(sentences)

    base_alpha = model.alpha
    base_min_alpha = model.min_alpha

    for mepoch in range(2):
        model.alpha = base_alpha 
        model.min_alpha = base_min_alpha
        for epoch in range(10):
            print('epoch',mepoch * 10 + epoch)
            model.train(sentences)
            model.alpha -= 0.002  # decrease the learning rate
            model.min_alpha = model.alpha  # fix the learning rate, no decay
        shuffle(sentences)

    # saving the model    
    model.save(model_file)
    print 'model trained and saved'
else:
    model = gensim.models.Doc2Vec.load(model_file)
    print 'model loaded'

model loaded


In [92]:
# 6 Tiny helper functions

def print_word_list(wl):
    str =  ' '.join(wl)
    pattern = re.compile('\s.\s')
    shift = 0
    for ma in pattern.finditer(str):
        str = str[:ma.start(0)-shift]+ma.group(0)[1:]+ str[ma.end(0)-shift:]
        shift +=1
    if str[-2] == ' ':
        str = str[:-2] + str[-1:]
    return str

def get_print(sentence_or_similar):
    if type(sentence_or_similar) is gensim.models.doc2vec.LabeledSentence:
        word_list = sentence_or_similar[0]
    elif type(sentence_or_similar) is int: # just an index
        word_list = sentences[sentence_or_similar][0]
    else: # TaggedDocument class
        word_list = sentences[int(sentence_or_similar[0][5:])][0]
    return print_word_list(word_list)

    
def get_index_tag(sentence):
    return sentence[1][0]

def get_index(sentence_or_similar):
    if type(sentence_or_similar) is gensim.models.doc2vec.LabeledSentence:
        return int(get_index_tag(sentence_or_similar)[5:])
    else:
        return int(sentence_or_similar[0][5:])
    
def equal_word_lists(index1, index2):
    wl1 = sentences[index1][0]
    wl2 = sentences[index2][0]
    if len(wl1) != len(wl2):
        return False
    else:
        for i in range(len(wl1)):
            if wl1[i] != wl2[i]:
                return False
    return True

def get_lab_sent_by_similar(similar):
    print get_index(similar)
    return sentences[get_index(similar)]

def get_similarity_by_index(index1, index2):
    return model.docvecs.similarity(index1,index2)

# HTML Helper
def pack_into_elem(tag, clazz, content):
    return '<' + tag + ' class="' + clazz + '"> ' + content+ ' </' + tag +'>'

pre = '''<style>
          .act {font-weight: bold}
          .i {color: grey}
          .sim {color: orange}
          .n {color: blue}
          .p {color: red}
          .r {color: green}          
     </style>'''

In [93]:
# 5 Test: printing sentence 9 and getting the most similar ones.
if do_tests:
    test_sentence_index = 2639533
    print get_print(test_sentence_index)
    sims = model.docvecs.most_similar('SENT_'+str(test_sentence_index),topn = 30)
    print 'similar sentence',len(sims)
    print '\nSIMILAR SENTENCES\n'
    for sim in sims:
        print get_print(sim),sim

New Haven, CT: Yale University Press.
similar sentence 30

SIMILAR SENTENCES

New Haven, CT: Yale University Press. ('SENT_2560974', 0.9662174582481384)
New Haven, CT: Yale University Press. ('SENT_2310654', 0.9655746221542358)
New Haven, CT: Yale University Press. ('SENT_2724632', 0.9649016857147217)
New Haven, CT: Yale University Press. ('SENT_2859928', 0.9643991589546204)
New Haven, CT: Yale University Press. ('SENT_1940547', 0.964377760887146)
New Haven, CT: Yale University Press. ('SENT_2130561', 0.9637207984924316)
New Haven, CT: Yale University Press. ('SENT_2957770', 0.9633453488349915)
New Haven, CT: Yale University Press. ('SENT_313266', 0.963297426700592)
New Brunswick, NJ: Rutgers University Press. ('SENT_4208346', 0.9628815650939941)
New Haven, CT: Yale University Press. ('SENT_2871939', 0.9628716707229614)
New Haven, CT: Yale University Press. ('SENT_2131374', 0.9623834490776062)
New Haven, CT: Yale University Press. ('SENT_44934', 0.9622904062271118)
New Haven, CT: Yale 

In [94]:
# 7 Test: iterate over similar sentences
# needs the sentences loaded (cell 2)
if do_tests:
    index = 1983
    # len(sentences)
    # print sentences[index]
    sentence = get_print(index)
    print sentence
    selected_indices = [index]

    for sentence in range(10):
        sims = model.docvecs.most_similar('SENT_'+str(index))
        while True:
            selected = random.choice(sims)
            check_index = int(selected[0][5:])
            if check_index not in selected_indices:
                break
        index = check_index
        selected_indices.append(index)
        print get_print(selected)

Here isa classic example of scientific discovery being concerned with the union of opposites: Kepler ’s laws and Galileo ’s laws being both reconciled and superseded bya principle which was both superior to both and yet comprehended both.
They also demonstrate both the importance and the failure of law to address this dilemma.
It seemed to many both too little and too late.
Solution This sketch demonstrates both lcd.ScrollDisplayLeft and lcd.ScrollDisplayRight.
Solution This sketch demonstrates both lcd.ScrollDisplayLeft and lcd.ScrollDisplayRight.
Still, both of you know what you are talking about.
Mr. Johnson sued both Mr. Womack and Hooters.
He sums up muchof whatis containedin both of the scatologicalimages thathavebeen examinedhere.
They routinely interact with both the courts and corrections.
It was both bathetically funny and extremely poignant.
Tolerance of Laws That Are Both Underinclusive and Overinclusive Many laws are both underinclusive and overinclusive.


In [95]:
# 8 Story Treenode class

PARENT = -1
QUIT = -2
NEXT = -3

num_similars = 10
num_random = 10

class LabSentTreeNode:
    
    def __init__(self, labeledSentence, parent = None):
        self.sentence = labeledSentence
        self.sentence_index = get_index(self.sentence)
        self.similars = self.get_similars()  
        self.randoms = self.get_randoms()
        self.children = {} # index: SentenceTreeNode
        self.selected_child = '' # None
        self.parent = parent
             
    def get_similars(self):
        return model.docvecs.most_similar(get_index_tag(self.sentence),topn = num_similars)
    
    def get_randoms(self):
        randoms = []
        for index in range(num_random):
            rnd_sen = sentences[random.randint(0,len(sentences))]
            randoms.append(rnd_sen)     
        return randoms
                
    def print_options(self):
        for index, sentence in enumerate(self.similars):
            add = '(*)' if get_similar_index(sentence) in self.children else ''
            print index, add, get_print(sentence), "%.3f" % sentence[1]
        if self.parent:
            print 'p: ', get_print(self.parent.sentence)
        if self.sentence_index < len(sentences) - 2:
            print 'n: ', get_print(sentences[self.sentence_index + 1])           
        for index,sentence in enumerate(self.randoms):
            print 'r'+str(index) +": ",  get_print(sentence)
            
    def get_options_html(self):
        html = ''
        for index, sentence in enumerate(self.similars):
            content = pack_into_elem('span','',get_print(sentence))
            index = pack_into_elem('span','',str(index)+': ')
            sentence_index = get_index(sentence)
            index += '❗️' if sentence_index in added_sentences else ''
            index_distance =  locale.format('%d', abs(self.sentence_index - sentence_index), 1)
            similar = pack_into_elem('span','sim',("%.3f" % sentence[1]) + ' / ' + str(index_distance))
            html += pack_into_elem('div', '', index + content + similar)
        if self.parent:
            content = pack_into_elem('span','',get_print(self.parent.sentence))
            index = pack_into_elem('span','','P: ')
            sentence_index = self.parent.sentence_index
            index_distance =  locale.format('%d', abs(self.sentence_index - sentence_index), 1)
            similarity = get_similarity_by_index(self.sentence_index, self.parent.sentence_index)
            similar = pack_into_elem('span','sim',("%.3f" % similarity) + ' / ' + str(index_distance))
            html += pack_into_elem('div', 'p', index + content + similar)
        if self.sentence_index < len(sentences) - 2:
            content = pack_into_elem('span','n',get_print(sentences[self.sentence_index + 1]))
            index = pack_into_elem('span','','N: ')
            sentence_index = self.sentence_index + 1
            index += '❗️' if sentence_index in added_sentences else ''
            index_distance =  locale.format('%d', abs(self.sentence_index - sentence_index), 1)
            similarity = get_similarity_by_index(self.sentence_index, self.sentence_index + 1)
            similar = pack_into_elem('span','sim',("%.3f" % similarity) + ' / ' + str(index_distance))
            html += pack_into_elem('div', 'n', index + content + similar)
        for index,sentence in enumerate(self.randoms):
            content = pack_into_elem('span','',get_print(sentence))
            index = pack_into_elem('span','','R'+str(index)+': ')
            sentence_index = get_index(sentence)
            index += '❗️' if sentence_index in added_sentences else ''
            index_distance =  locale.format('%d', abs(self.sentence_index - sentence_index), 1)
            index_distance =  locale.format('%d', abs(self.sentence_index - sentence_index), 1)
            similarity = get_similarity_by_index(self.sentence_index, get_index(sentence))
            similar = pack_into_elem('span','sim',("%.3f" % similarity) + ' / ' + str(index_distance))
            html += pack_into_elem('div', 'r', index + content + similar)
        html += pack_into_elem('div', '', 'Q: Quit 💣')
        return html
    
    def get_sentence_html(self):
        return pack_into_elem('p', 'act', get_print(self.sentence))
    
    def select_child(self):
        u_input = raw_input('Next child: ')
        if u_input == 'p':
            selected_index = PARENT
        elif u_input == 'q':
            return None
        elif u_input == 'n':
            selected_index = NEXT
        elif u_input.startswith('r'):
            selected_index = 100 + int(u_input[1:])
        else:
            try:
                selected_index = int(u_input)
            except ValueError:
                return self
        if selected_index >= 0 and selected_index < len(self.similars):
            lab_sent = get_lab_sent_by_similar(self.similars[selected_index])
            child =  LabSentTreeNode(lab_sent, self)
            self.children[u_input] = child
            self.selected_child = u_input
            return child
        elif selected_index >= 100 and selected_index < len(self.randoms) + 100:
            #print 'random sen'
            child =  LabSentTreeNode(self.randoms[selected_index - 100], self)
            self.children[u_input] = child
            self.selected_child = u_input   
            return child
        elif selected_index == PARENT and self.parent:
            return self.parent
        elif selected_index == NEXT:
            child =  LabSentTreeNode(sentences[self.sentence_index + 1], self)
            self.children[u_input] = child
            self.selected_child = u_input   
            return child
        # a weird number
        return self
        
    def toJSON(self):
        children_toJSON = {}
        for child_index in self.children:
            children_toJSON[child_index] = self.children[child_index].toJSON()
            
        return {'sentence':get_print(self.sentence),
                'index':get_index(self.sentence),
               'children':children_toJSON,
                'selected_child':self.selected_child
               }  

In [96]:
# 8 Story creator/log helper functions

def get_story(root_node):
    act_sentence = root_node
    story = []
    while act_sentence:
        story.append(get_print(act_sentence.sentence))
        if act_sentence.selected_child  != '':
            act_sentence = act_sentence.children[act_sentence.selected_child]
        else:
            break
    return story

def log_json(root_node):
    with open('log.json','w') as output:
        output.write(json.dumps(root_node.toJSON(),indent=2))
    
def log_story(root_node):
    story = get_story(root_node)
    with open('story.txt','w') as output:
        for l in story:
            output.write(l+'\n')   
    
def print_story(root_node):
    story = get_story(root_node)
    for l in story:
        print(l)   

def dump_story(root_node):
    with open('story.dump','w') as dump_file:
        pickle.dump(root_node,dump_file)

In [97]:
# 9 Story creator

load_from_log = True

added_sentences = set()  

if load_from_log:
    with open('story.dump','r') as in_file:    
        root_node = pickle.load(in_file)
        actual_node = root_node
        while actual_node.selected_child != '':
            added_sentences.add(actual_node.sentence_index)
            actual_node = actual_node.children[actual_node.selected_child]
            
else:  
    sentence = sentences[random.randint(0,len(sentences))]
    root_node = LabSentTreeNode(sentence)
    actual_node = root_node

while actual_node:
    clear_output()
    log_json(root_node)
    log_story(root_node)
    dump_story(root_node)
    added_sentences.add(actual_node.sentence_index)
    display(HTML(pre + actual_node.get_sentence_html() + actual_node.get_options_html()))
    time.sleep(0.4)
    actual_node = actual_node.select_child()
clear_output()
print '📖 ⭐ 📖'
print_story(root_node)
print '👋🏽'

📖 ⭐ 📖
Islamic thought reciprocated by dividing the worla into dar-a/-Islam, the region of submission to the will of God.
To suppose God, it will be said, is to deny him.
He said what Scaring wanted him to say.
I do n't know how to say it.
If they say that they believe something—they do not.
After all, you say toe-may-toe, andI say toe-ma-toe.
At the same time, it is to say much more.
I don ’t think there is much more to say.
I do n't know how to say it.
ButI believe that the reality- the truth thatI recognize in suffering asI do n't in comfort and happiness- that the reality of pain is not pain.
That is why we speak of truth, not reality.
But the honest truth is that we really do not know.
Therefore, if you tell people the truth, they are not going to accept what you do.
There isa lot of truth to that.
Th e doctor 's wife told me the truth.
Thou shalt deny God and love truth; thereforeI am an atheist.
Th e doctor 's wife told me the truth.
Takea look at the following truth table.
Next 

In [ ]:
# TRYING TO TRAIN MORE
# RUNS BUT DOESNT EXTEND THE MODEL FILE
print len(model.docvecs)
line = 'Therefore, if you tell me the truth, they are not going to reject what you say.'
ls = gensim.models.doc2vec.LabeledSentence(words=line.split(), tags=['SENT_%s' % len(sentences)])
sentences.append(ls)
model.train([ls])
print len(model.docvecs)
print sentences[len(sentences)-1]
get_similarity_by_index(len(sentences)-1,400)

### Findings
next to similarity(d1,d2) there is also n_similarity(ds1, ds2), which takes lists.
```
s_index = 400
s = sentences[s_index]
res = model.docvecs.most_similar(get_index(s))
# print res
for s in res:
    print get_print(s)
    print s[1]
    index = get_index(s)
    print get_similarity_by_index(s_index,index)
    print model.docvecs.n_similarity([s_index],[index])
    print '######'
    
```
resulting in:
```
0.570710778236
0.570710771218
0.570710771218
######
I have only been twice; so many things bewildered me.
0.563167154789
0.563167148085
0.563167148085
######
```

In [110]:
html = '''
<script>
var voices = synth.getVoices();
for(var i = 0 ;i < voices.length; i++){
console.log(voices[i].name);
}

</script>
'''
display(HTML(html))



In [117]:
pitch = 1.5
rate = 1.0

text = get_story(root_node)
var = 'var text= "'+' '.join(text) + '";'


html = '''
<script>
var synth = window.speechSynthesis;
var   voices = synth.getVoices();

'''+ var +'''
  var utterThis = new SpeechSynthesisUtterance(text);
  var selectedOption = "Agnes"
  for(i = 0; i < voices.length ; i++) {
    if(voices[i].name === selectedOption) {
      utterThis.voice = voices[i];
    }
  }
  utterThis.pitch = '''+str(pitch)+''';
  utterThis.rate = '''+str(rate)+''';
  synth.speak(utterThis);

</script>
'''
display(HTML(html))